In [ ]:
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from psycopg2.extensions import AsIs
import os

In [ ]:
excelFileName="NonSM_CustomerPM_20230410.xlsx"
if os.path.exists(excelFileName)==False:
    raise Exception(f"Not found file {excelFileName} on this folder")
else:
    print(f"{excelFileName} is master file to get company data to extract into system")

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
    

def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)     
    # print(sql)
    # print(params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 




def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
    

def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)     
    # print(sql)
    # print(params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 




def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
    

def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)     
    # print(sql)
    # print(params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 




def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()

In [ ]:
dfCompForAdmin=pd.read_excel(excelFileName)
dfCompForAdmin.columns=['company_name','company_full_name','company_address']
dfCompForAdmin['company_telephone']='-'
dfCompForAdmin['is_customer']=True
dfCompForAdmin['is_subcontractor']=False
dfCompForAdmin['is_managed_by_admin']=True

# check existing company name
dfCompForAdmin['upper_name']=dfCompForAdmin['company_name'].apply( lambda x : str(x).upper() )

print(dfCompForAdmin.info())
dfCompForAdmin.head()

In [ ]:
sql_level=""" 
select UPPER(company_name) as company_name from app_company 
"""
df_all_comp_name=list_data(sql_level,None,get_postgres_conn())
# print(df_all_comp_name.info())
listExistingComp=df_all_comp_name['company_name'].tolist()

dfExistingComp=dfCompForAdmin[dfCompForAdmin['upper_name'].isin(listExistingComp)]

dfCompForAdmin=dfCompForAdmin[~dfCompForAdmin['upper_name'].isin(listExistingComp)]
dfCompForAdmin=dfCompForAdmin.drop(columns=['upper_name'])

print(dfCompForAdmin.info())
dfExistingComp.to_csv('Existing_Company.csv',index=False)
dfExistingComp


In [ ]:
result=add_data_values(dfCompForAdmin,'app_company',get_postgres_conn())
if  result==1:
    print(f"{len(dfCompForAdmin.index)} items have been imported to database successfully.")
    print("importing data succeeded")

# List new company

In [ ]:
listNewAdminComp=dfCompForAdmin['company_name'].tolist()
# listNewAdminComp
# listNewAdminComp=['CISCO','DELL']

# sql_admin_comp=f"""  select id,company_name from app_company where company_name = %(comp_param)s  """
# dfX=list_data(sql_admin_comp,{"comp_param": 'YIP'},get_postgres_conn())



sql_admin_comp=f"""  select id,company_name from app_company where company_name  in  %(comp_param)s  order by id """
df_new_company=list_data(sql_admin_comp,{"comp_param":tuple(listNewAdminComp)},get_postgres_conn())

print(df_new_company.info())
df_new_company.to_csv('1-NewAdminCompany.csv',index=False)